This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [6]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'XXXXXXXXXXXX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [19]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?limit=1&api_key='+API_KEY)

r.json()


{'dataset': {'collapse': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'data': [['2019-03-13',
    None,
    78.15,
    77.25,
    78.15,
    None,
    66050.0,
    5149125.0,
    None,
    None,
    None]],
  'database_code': 'FSE',
  'database_id': 6129,
  'dataset_code': 'AFX_X',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'end_date': '2019-03-13',
  'frequency': 'daily',
  'id': 10095370,
  'limit': 1,
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'newest_available_date': '2019-03-13',
  'oldest_available_date': '2000-06-07',
  'order': None,
  'premium': False,
  'refreshed_at': '2019-03-13T22:34:24.527Z',
  'start_date': '2000-06-07',
  'transform': None,
  'type': 'Time Series'}}

In [24]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

print(r.json()['dataset']['description'])
print(r.json()['dataset']['column_names'])
print(r.json()['dataset']['data'])


Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
[['2019-03-13', None, 78.15, 77.25, 78.15, None, 66050.0, 5149125.0, None, None, None]]


These are your tasks for this mini project:


In [45]:
# 1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, 
# for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)


In [52]:
# 2. Convert the returned JSON object into a Python dictionary.

# NOTE: r.json() is already a Python dictionary, 
#       so not sure the meaning of this request. 

# Convert the stock series data into a useable dict.
colnames = r.json()['dataset']['column_names']
stockseries = dict()
for i,colname in enumerate(colnames):
    stockseries[colname] = [lst[i] for lst in r.json()['dataset']['data'][:] ]
print(stockseries.keys())

dict_keys(['Date', 'Daily Traded Units', 'Traded Volume', 'Close', 'Turnover', 'High', 'Daily Turnover', 'Change', 'Open', 'Last Price of the Day', 'Low'])


In [70]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

# convert Nones to nans so I can do the math
opn = list(map(lambda x: float('nan') if x==None else x, stockseries['Open']))

print('Open prices: lowest = %1.3f, highest = %1.3f'%(min(opn), max(opn) ) )

Open prices: lowest = 34.000, highest = 53.110


In [82]:
# 4. What was the largest change in any one day (based on High and Low price)?

# convert Nones to nans so I can do the math. 
# Use set instead of list to allow subtraction
low = set(map(lambda x: float('nan') if x==None else x, stockseries['Low']))
high = set(map(lambda x: float('nan') if x==None else x, stockseries['High']))

print('Largest daily change = %1.3f'%(max(high-low)))

Largest daily change = 53.540


In [81]:
# 5. What was the largest change between any two days (based on Closing Price)?

# convert Nones to nans so I can do the math. 
closing = list(map(lambda x: float('nan') if x==None else x, stockseries['Close']))

# use max key function to get max abs value
print('Largest two day change = %1.3f'%(max(set(closing[1:]) - set(closing[:-1]), key=abs)))


Largest two day change = 35.800


In [83]:
# 6. What was the average daily trading volume during this year?

# convert Nones to nans so I can do the math. 
volume = list(map(lambda x: float('nan') if x==None else x, stockseries['Traded Volume']))

print('Mean volume = %1.3f'%(sum(volume)/len(volume)))

Mean volume = 89124.337


In [105]:
# 7. (Optional) What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)

# NOTE: this median code only works for odd sample numbers, like here (255)
print('Median volume = %1.3f'%(sorted(volume)[int(len(volume)/2)]))

Median volume = 76286.000
